**Emoji Recognition With web cam**

*The necessary imports*

In [ ]:
import cv2
import imutils
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import imageio as ig


*Loading the model, labels and emoji pictures*

In [ ]:
model = load_model('emojirecog.hdf5')    #pre-trained model
bg = None
cam = cv2.VideoCapture(0)
top, right, bottom, left = 80, 400, 275, 640     #specifying emoji box location
ra_weight = 1
labels = ['call_me','fingers_crossed','index_up','okay','paper','rock','rock_on','scissor','spock','thumbs_up']   # emoji labels for identification
emojis = []
for i in labels:
    ran = cv2.imread("/emojis/" + i + ".png",cv2.COLOR_BGR2RGB)    #to read each of the 10 emoji pictures using the labels 
    emojis.append(np.array(ran))
    
emojis = np.array(emojis)      # the emoji's pictures 

*Function to form back ground image for comparison from the first 50 frames*

In [ ]:
def run_avg(image, ra_weight):
    global bg
    if bg is None:
        bg = image.copy().astype("float")
        return
    cv2.accumulateWeighted(image, bg, ra_weight)  #calculates the running average for the frames

*Function to form the subtracted image from hand and background*

In [ ]:
def segment(image):
    global bg
    diff = cv2.absdiff(image,bg.astype("uint8"))    #difference between background and current frame
    threshold = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)[1]        #keep pixels only within threshold
    cnts = cv2.findContours(threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]    #finding the contours from the modified frame
    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        return (threshold, segmented) 

*Function*

In [ ]:
while True:
    ret, frame = cam.read() 
    frame = imutils.resize(frame, width=800)
    frame = cv2.flip(frame, 1)
    clone = frame.copy()
    roi = frame[top:bottom, right:left]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY) #converting the coloured image to BGR2GRAY
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    number = "Hello"
    if frame_no < 50: #for first 50 frames running average is executed
        run_avg(gray, ra_weight)
    else:
        hand = segment(gray)
        if hand is not None:
            (threshold, segmented) = hand
            cv2.drawContours(clone, [segmented + (right, top)], -1, (255, 255, 0)) #right and top are to move the contours from (0,0)
            im = Image.fromarray(threshold)
            im = im.resize((240,200),Image.ANTIALIAS)
            im = np.array(im)
            im = np.expand_dims(im,axis = 2)
            im = np.expand_dims(im,axis = 0)
            cv2.imshow("emojis",emojis[model.predict_classes(im)[0]]) #model.predict returns a string with emoji name and extracts the emoji with that string to print it
    cv2.rectangle(clone, (left, top), (right, bottom), (0,0,0), 2)
    cv2.putText(clone,number,(0,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    cv2.imshow("Video Feed", clone)
    k = cv2.waitKey(1)
    
    
cam.release()
cv2.destroyAllWindows()